In [1]:
from typing import List, Dict, Tuple, Set
import numpy as np
import time
from dataclasses import dataclass
import time
from datetime import date
from collections import deque
import pandas as pd
from pathlib import Path
from enum import Enum
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from io import StringIO

# CHROME_WEBDRIVER_PATH = "/home/fabrice/Documents/PROGRAMMING/ChromeWebDrive/chromedriver"
CHROME_WEBDRIVER_PATH = "/usr/lib/chromium-browser/chromedriver"

# For retries
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Local files.
import constants as c
from constants import C_all as C
from website_crawler import WebCrawler

In [2]:
# %pip install selenium

In [3]:
# %pip install html5lib

# Data Grabber

## Get an unprocessed url

In [4]:
df_competition_links = pd.read_csv("results/competition_links.csv")
df_competition_links

,Tournament link,Base url,Crawl date,Tournament id,Processed
0,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,9,0
1,https://ergebnisse.dancecomp.de/2022/28-0207_w...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,10,0
2,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,11,0
3,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,12,0
4,https://www.gruen-weiss-aachen.de/files/ergebn...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,13,0
...,...,...,...,...,...
218,https://www.hessen-tanzt.de/media/ht2023/56-13...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,227,0
219,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,228,0
220,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,229,0
221,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,230,0


In [5]:
print("URLs of already processed competitions:")
df_competition_links.loc[df_competition_links['Processed'] == 1]

URLs of already processed competitions:


,Tournament link,Base url,Crawl date,Tournament id,Processed


In [6]:
n = len(df_competition_links)
n_processed = len(df_competition_links.loc[df_competition_links['Processed'] == 1])
print(f"Nr. of crawled   competitions:  {n}\n"\
      f"Nr. of processed competitions:  {n_processed}\t({n_processed/n*100:0.2f} %)")

Nr. of crawled   competitions:  223
Nr. of processed competitions:  0	(0.00 %)


In [7]:
# Select the links and the ids of unprocessed tournaments.
unprocessed_tournaments_df = df_competition_links[df_competition_links['Processed'] == 0][[C.TOUR_LINK, C.ID]]

for competition_url, competition_id in unprocessed_tournaments_df.itertuples(index=False):
    print(f"Id: {competition_id}\tURL: {competition_url}")

Id: 9	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/3le-sen1sstd/index.htm
Id: 10	URL: https://ergebnisse.dancecomp.de/2022/28-0207_wdsfopenstdsen3/index.htm
Id: 11	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2022/dm_hgrsstd/index.htm
Id: 12	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/glinde-sen3sstd/index.htm
Id: 13	URL: https://www.gruen-weiss-aachen.de/files/ergebnisse/2023/2023-06-3le/3-1706_rl_hgrslat/index.htm
Id: 14	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/g55-2/index.htm
Id: 15	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/l66-4/index.htm
Id: 16	URL: https://berlin-dance-festival.de/files/bdf/results/2023/5-0104_wdsfopenstdjun2/index.htm
Id: 17	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2020/02-22.Deutsche.Meisterschaft.Junioren.II.B.Latein/index.htm
Id: 18	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/dpsen3slat/index.htm
Id: 19	URL: https://www.tanzsport.de/files/

## Crawl the data directly from the source code of the competition website

In [8]:
url_id = 2

Tested for url id indices:
- 0

In [9]:
is_english: bool = False
competition_url: str = unprocessed_tournaments_df[C.TOUR_LINK][url_id]
competition_id: int  = unprocessed_tournaments_df[C.ID][url_id]
# Start a Chrome driver to open the selected url.
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(competition_url)

# From the tournament link - extract the competition links.
# menu_url = re.compile('src="(.*)"').search(html).group(1)
competition_menue_links = [competition_url.replace('index.htm', "menu.htm#id2")]
df_tables = []
for l in competition_menue_links:
    driver.get(l)
    # Wait a second since the menu has to build itself before it can be collected.
    time.sleep(1)
        
    html: str = driver.page_source
    comp_date: str = None
    comp_class: str = None
    comp_title: str = None
    
    # Extract the title, class and date of the competition.
    # German format:    dd.mm.yyyy
    # English format:   dd/Mon/yyyy
    comp_date_and_class: str = driver.title
    match_date_de = re.search(r'\b(\d{2}\.\d{2}.\d{4})\b', comp_date_and_class)
    match_date_en = re.search(r'(\d{2}/[a-zA-Z]+/\d{4})', comp_date_and_class)
    if match_date_de:
        comp_date = match_date_de.group(1)
    elif match_date_en:
        comp_date = match_date_en.group(1)
        is_english = True
    comp_class = comp_date_and_class.replace(f"{comp_date} ", '')
    assert comp_date is not None, f"Unknown html! Could not extract competition date from {comp_date_and_class}."
    assert comp_class is not None, f"Unknown html! Could not extract competition class from {comp_date_and_class}."

    match_title = re.search(r'<td>(.*?)</td>', html)
    if match_title:
        comp_title = match_title.group(1)
    assert comp_title is not None, "Unknown html! Could not find competition name!"

    df_tables.append(pd.read_html(StringIO(html)))

# It may be that the data is stored in a nested DataFrame. 
# In this case extract it.
if len(df_tables) == 1:
    df_tables = df_tables[0]

# Since the download is complete now, close the driver.
driver.close()

print(competition_url)
df_tables

https://www.tanzsport.de/files/tanzsport/ergebnisse/2022/dm_hgrsstd/index.htm


[                                             0  1
 0  Deutsche Meisterschaft Standard - Bielefeld  ≡,
                    0                                                  1
 0      Veranstalter:                         Deutscher Tanzsportverband
 1        Ausrichter:                            Bielefelder TC Metropol
 2     Turnierleiter:            Reichling, Daniel TSC Brühl im BTV 1879
 3         Beisitzer:                     Münster, Ivo TTC Rot-Gold Köln
 4       Chairperson:           Sónyi, Markus Grün-Gold-Casino Wuppertal
 5         Protokoll:  Kirsten, Mario Tanzsportgemeinschaft Bünde Kno...
 6   Wertungsrichter:                                                NaN
 7                 A:                                    Degroot, Cedric
 8                 B:               Glohr, Norbert Imperial-Club Hamburg
 9                 C:            Hötting, Marc-Oliver TSK Sankt Augustin
 10                D:       Kies, Christoph Tanzsportclub Casino Dresden
 11                E:

In [10]:
# TODO: If englisch: 
C = c.constants_in_language.get('de')
if is_english:
    C = c.constants_in_language.get('en')
    print("Is english!")

In [11]:
print(f"Title:\t{comp_title}")
print(f"Date:\t{comp_date}")
print(f"Name:\t{comp_class}")

Title:	Deutsche Meisterschaft Standard - Bielefeld
Date:	05.11.2022
Name:	Hgr. S Standard


In [12]:
# Create a folder to store the tables. Save a table and an equivalent README.md, containing the website link and the crawl date.
competition_dir = Path('results', 'comp_tables', f"{competition_id}")
# If directories do not exist, create them.
competition_dir.mkdir(parents=True, exist_ok=True)

# Save key information into a DataFrame and save it in the dir.
crawl_date = date.today().strftime("%B %d, %Y")
df_readme = pd.DataFrame({'Competition url': [competition_url],
                          'Crawl date': [crawl_date]})
df_readme.to_csv(Path(competition_dir, 'README.csv'))
# Save key information into a Markdown file and save it in the dir.
with open(Path(competition_dir, 'README.md'), 'w') as f:
    f.write(f'[Competition URL]({competition_url})\n')
    f.write(f'Crawled on {crawl_date}\n')
    f.write('by Fabrice Beaumont')

In [13]:
# Parse all tables.
# List of possible data tables:
class data_dfs(Enum):
    general_info: str   = 'General informations'
    adjudicators: str   = 'Adjudicators'
    couples: str        = 'Couples'
    ranking_list: str   = 'Ranking list'
    finals: str         = 'Finals'
    qualifications: str = 'Qualifications'

df_general_info: pd.DataFrame   = None
df_couples: pd.DataFrame        = None
df_finals: pd.DataFrame         = None
df_dict: Dict[str, List[pd.DataFrame]] = {
    data_dfs.general_info: [],
    data_dfs.adjudicators: None,
    data_dfs.couples: [],
    data_dfs.ranking_list: [],
    data_dfs.finals: [],
    data_dfs.qualifications: [],
}

# The first table is expected to be the competiton name in all cases.
competition_name = df_tables[0][0][0]

for i in range(1, len(df_tables)):
    table = df_tables[i]

    # Test for different criteria to sort the tables into the dictionary.
    # GENERAL INFORMATION table.
    if table[0][0] in C.KEY_ORGANIZER_OPTIONS:
        print(f"Found general info\t\tin table {i}")
        df_dict[data_dfs.general_info].append(table)
        continue
    
    # RANKING LIST table - finals section.
    if len(table[0]) > 1 and str(table[0][0]) in ['Endrunde', 'Final']:
        print(f"Found ranking list (finals)\tin table {i}")
        df_dict[data_dfs.ranking_list].append(table)
        continue

    # RANKING LIST table - all qualification rounds sections.
    if len(table[0]) > 1 and str(table[0][1])[-5:] in ["runde"]:
        print(f"Found ranking list (round)\tin table {i}")
        df_dict[data_dfs.ranking_list].append(table)
        continue
    
    # FINALS table (containing the scating system and placements for each couple from each adjudicator).
    if len(table[0]) > 1 and str(table[1][0]) in ['Wertungsrichter']:
        print(f"Found finals scating\t\tin table {i}")
        df_dict[data_dfs.finals].append(table)
        continue
    
    # QUALIFICATIONS table (containing points for all couples).
    if len(table[0]) > 1 and str(table[0][1]) in ['Wertungsrichter']:
        print(f"Found qualifications\t\tin table {i}")
        df_dict[data_dfs.qualifications].append(table)
        continue

    print(table)

Found general info		in table 1
Found ranking list (finals)	in table 2
Found ranking list (round)	in table 3
          0                                                  1    2        3   \
0      Platz                                          Paar Club   Nr        R   
1      Platz                                          Paar Club   Nr        R   
2        NaN                                                NaN  NaN      NaN   
3         1.  Tomas Fainsil / Violetta Fainsil TSC Astoria S...   13  F 3 2 1   
4        NaN                                                NaN  NaN      NaN   
..       ...                                                ...  ...      ...   
76       NaN                                                NaN  NaN      NaN   
77  38.- 39.  Jens Kothe / Vanessa Gergert Tanzsportzentrum ...   28        1   
78       NaN                                                NaN  NaN      NaN   
79  38.- 39.  Patrick Vrielmann / Marit Vrielmann TTC Gelb-W...   47        1   
8

## Processing of the general information and adjudicator list

In [14]:
df_general_info = df_dict[data_dfs.general_info][0].copy()
# Split the gereral information table into the tournament information and the adjudicator list.
split_id = np.where(df_general_info.iloc[:, 1].isnull() == True)[0][0]
# Extract the remaining rows as information about the adjudicators.
adjudicator_names = list(df_general_info.iloc[split_id+1:, 1])
adjudicator_ids = [x[:1] for x in list(df_general_info.iloc[split_id+1:, 0])]
adjudicators = dict(zip(adjudicator_ids, adjudicator_names))

# Extract the data into individual variables.
df_general_info = pd.DataFrame(df_general_info.iloc[:split_id, :])
df_general_info.set_index([0])
df_general_info.columns = [C.CATEGORY, C.VALUE]
df_general_info = df_general_info[1:]
# Remove colon in the category column if present.
df_general_info[C.CATEGORY] = df_general_info[C.CATEGORY].str.replace(':', '')

# Add more data to the general info.
additional_info: Dict[str, str] = {}
additional_info['Date'] = comp_date
additional_info['Title'] = comp_title
additional_info['Class'] = comp_class

df_general_info

,Category,Value
1,Ausrichter,Bielefelder TC Metropol
2,Turnierleiter,"Reichling, Daniel TSC Brühl im BTV 1879"
3,Beisitzer,"Münster, Ivo TTC Rot-Gold Köln"
4,Chairperson,"Sónyi, Markus Grün-Gold-Casino Wuppertal"
5,Protokoll,"Kirsten, Mario Tanzsportgemeinschaft Bünde Kno..."


In [15]:
# Process the dict of adjudicators.
adjudicators_id_club: Dict[str, Tuple[Tuple[str, str], str]] = dict()
for adjudicator_id, adjudicator_and_club in adjudicators.items():
    ac = adjudicator_and_club.split(" ")
    adjudicator, club = tuple(ac[0:2]), " ".join(ac[2:])
    
    adjudicators_id_club[adjudicator_id] = tuple([adjudicator, club])

# Create a DataFrame to hold the adjudicator information.
df_adjudicators = pd.DataFrame(columns=[C.SURNAME, C.NAME, C.CLUB])
# Add adjudicators which are unknown (defined as surname, name, club triple).
for _, adjudicator_data in adjudicators_id_club.items():
    name, surname = adjudicator_data[0]
    club = adjudicator_data[1]
    
    # Check if adjudicator is already known:    
    x = df_adjudicators.loc[(df_adjudicators[C.SURNAME] == surname) & (df_adjudicators[C.NAME] == name) & (df_adjudicators[C.CLUB] == club)]
    # Only add, if not already known.
    if len(x) == 0:   
        new_row = pd.DataFrame({C.SURNAME:surname, C.NAME:name, C.CLUB:club}, index=[0])
        df_adjudicators = pd.concat([new_row,df_adjudicators.loc[:]]).reset_index(drop=True)

df_dict[data_dfs.adjudicators] = df_adjudicators
df_dict[data_dfs.adjudicators].to_csv(Path(competition_dir, 'adjudicators.csv'))
df_dict[data_dfs.adjudicators]

,Surname,Name,Club
0,Olaf,"Schlemminger,",TSG d. TuS Komet Arsten
1,Zoltan,"Sandor,",
2,Prof.,"Pöhlau,",Dr. Frank TSC Rot-Gold-Casino Nürnberg
3,Euvgenia,"Parakhina,",
4,Rainer,"Kopf,",TSC Grün-Gold Speyer
5,Christoph,"Kies,",Tanzsportclub Casino Dresden
6,Marc-Oliver,"Hötting,",TSK Sankt Augustin
7,Norbert,"Glohr,",Imperial-Club Hamburg
8,Cedric,"Degroot,",


In [16]:
nr_adjudicators = len(df_dict[data_dfs.adjudicators])
nr_adjudicators

9

## Processing of the coupes list

In [17]:
if len(df_dict[data_dfs.couples]) > 0:
    df_couples = df_dict[data_dfs.couples]
    # Use the first row as column names and then delete it.
    df_couples.columns = df_couples.iloc[0]
    df_couples = df_couples[1:]
    df_couples = df_couples.set_index(C.NR)
    # Extract the names of the lady and the man individually.
    # Do not separate the fist and last names since depending on multiple name and titles (e.g. 'Dr.', 'Prof.') this may be more complicated.
    # To identify a person simply test if the surname is part of the name string.
    df_couples[[C.MAN, C.LADY]] = df_couples[C.COUPLE].str.split(" / ", expand = True)
    # Rearrange the columns.
    df_couples = df_couples[[C.MAN, C.LADY, C.COUPLE, C.CLUB]]
    df_dict[data_dfs.couples] = df_couples

df_dict[data_dfs.couples]

[]

## Processing of the ranking list

In [18]:
df_dict[data_dfs.ranking_list][0]

,0,1,2,3,4,5,6,7
0,Endrunde,Endrunde,Endrunde,Endrunde,Endrunde,Endrunde,Endrunde,Endrunde
1,Platz,Paar/Club,LW,TG,WW,SF,QU,PZ
2,1.,Tomas Fainsil / Violetta Fainsil (13) TSC Asto...,111111111 1.0,111111111 1.0,111111111 1.0,111111111 1.0,111111111 1.0,5.0
3,2.,Dominik Stöckl / Anna Gommer (43) Bielefelder ...,222342234 2.0,324344244 4.0,242353234 3.0,222352234 2.0,223353244 2.0,13.0
4,3.,Erik Kem / Viktoria Grusevskaja (24) Tanzsport...,343224523 3.0,443225433 3.0,323222523 2.0,333223523 3.0,434225423 3.0,14.0
5,4.,Emanuele Cannistraro / Anna Lubianetska (6) Sc...,534636652 5.0,232632522 2.0,434636442 4.0,544635452 4.0,342634632 4.0,19.0
6,5.,Lukasz Switalski / Natalia Mikolajczyk (45) Ta...,455463446 4.0,555463656 5.0,555464656 5.0,455464646 5.0,555462556 5.0,24.0
7,6.,William Lauth / Julia Maria Scherer (30) TSC L...,666555365 6.0,666556365 6.0,666545365 6.0,666546365 6.0,666546365 6.0,30.0


In [19]:
df_ranking_finals = df_dict[data_dfs.ranking_list][0].copy()
# Drop the first row which contains only the key word 'Finals'.
df_ranking_finals = df_ranking_finals[1:]
# Set the first row as headder, and drop it form the data body.
df_ranking_finals.columns = df_ranking_finals.iloc[0]
df_ranking_finals = df_ranking_finals[1:]

# Since the placements of the finals are redundant information and will be extracted from the finals table,
# delete it here. In case that there were more rounds this allows to easily join the other rankings to this table.
df_ranking_finals.drop(columns=C.get_dancenames_short() + ['PZ'], inplace=True, errors='ignore')
# Split the club name from the couples name (and their number).
df_ranking_finals[[C.COUPLE, C.CLUB]] = df_ranking_finals['Paar/Club'].str.split(")", expand = True)
df_ranking_finals.drop(columns=['Paar/Club'], inplace=True)
df_ranking_finals[[C.COUPLE, C.NR]] = df_ranking_finals[C.COUPLE].str.split("(", expand = True)
df_ranking_finals[[C.ROUND]] = C.KEY_FINAL
df_ranking_finals = df_ranking_finals[[C.PLACEMENT, C.ROUND, C.NR, C.COUPLE, C.CLUB]]
df_ranking_finals

AttributeError: type object 'C_de' has no attribute 'KEYWORD_FINAL'

In [ ]:
df_ranking_other = df_dict[data_dfs.ranking_list][1].copy()
# Drop all rows which contain only NaN values. These are originally used to separate the tables.
df_ranking_other.dropna(axis=0, how='all', inplace=True)
# Rename the columns.
df_ranking_other.columns = [C.PLACEMENT, C.COUPLE, C.CLUB]

# Split the club name from the couples name (and their number).
df_ranking_other[[C.NR]] = df_ranking_other[C.COUPLE].str.extract(r'\((\d*)\)', expand = True)
df_ranking_other[[C.COUPLE]] = df_ranking_other[C.COUPLE].str.extract(r'(.*)\(', expand = True)

# Collect the row indices that contain names instead of placements. These are the round names.
round_rows = df_ranking_other[df_ranking_other[C.PLACEMENT].str.endswith('runde')]
nr_rounds: int = len(round_rows)
round_list: List[Tuple[int, str]] = [(i, round_rows[C.PLACEMENT][i]) for i in round_rows.index] + [(len(df_ranking_other), '')]

# Mark all following rows with the corresponding round name in a new column.
df_ranking_other[[C.ROUND]] = round_list[0][1]
for i in range(len(round_list) - 1):
    start_index, round_name = round_list[i]
    end_index = round_list[i+1][0]
    df_ranking_other.loc[start_index : end_index + 1, C.ROUND] = round_name


# Now delete the rows with the round names. 
# Notice that we ignore the last entry since this is an artificial one to finish the for-loop above.
for i, _ in round_list[:-1]:
    df_ranking_other.drop(i, inplace=True)

# Rearrange the column order.
df_ranking_other = df_ranking_other[[C.PLACEMENT, C.Round, C.NR, C.COUPLE, C.CLUB]]

df_ranking_other

AttributeError: type object 'C_de' has no attribute 'Round'

In [ ]:
df_rankings = pd.concat([df_ranking_finals, df_ranking_other])
nr_couples: int = len(df_rankings)
# Set the couples number as row index.
df_rankings.set_index(C.NR, inplace=True)
df_dict[data_dfs.ranking_list] = df_rankings
df_dict[data_dfs.ranking_list].to_csv(Path(competition_dir, c.filenames.ranking_lst.value))
df_dict[data_dfs.ranking_list]

,Platz,Paar,Club,Round
Nr.,,,,
13,1.,Tomas Fainsil / Violetta Fainsil,TSC Astoria Stuttgart,NaN
43,2.,Dominik Stöckl / Anna Gommer,Bielefelder TC Metropol,NaN
24,3.,Erik Kem / Viktoria Grusevskaja,Tanzsportclub Dortmund,NaN
6,4.,Emanuele Cannistraro / Anna Lubianetska,Schwarz-Weiß-Club Pforzheim,NaN
45,5.,Lukasz Switalski / Natalia Mikolajczyk,Tanzsportzentrum Dresden,NaN
30,6.,William Lauth / Julia Maria Scherer,TSC Landau i.d. Pfalz,NaN
7,7.,Matteo Cesaretti / Emily Matthies,Grün-Gold-Club Bremen,2. Zwischenrunde
48,8.,Mykyta Zherdiev / Fabien Lax,TSC Rot-Gold-Casino Nürnberg,2. Zwischenrunde
2,9.,Philip Andraus / Ekaterina Tsvetkova,TTC Fortis Nova Maintal,2. Zwischenrunde


## Processing of the finals

In [ ]:
df_finals = df_dict[data_dfs.finals][0]
dance_name_rows = df_finals[df_finals[0].isna()]
dance_name_ids = dance_name_rows.index.to_list() + [len(df_finals.index)]

# Add a column to display the dance for which the rankings are for. 
# This will replace the horizonal headlines indicating the dances - which will be deleted afterwards.
df_finals[[C.DANCE]] = ''
df_finals[C.DANCE].iloc[1] = C.DANCE
for i in range(len(dance_name_ids) - 1):
    start_index = dance_name_ids[i]
    dance_name = c.parse_dance_name(dance_name_rows.loc[start_index, 1], C)
    end_index = dance_name_ids[i+1] - 1
    df_finals.loc[start_index:end_index, C.DANCE] = dance_name

# Now delete the rows with the dance names. 
# Notice that we ignore the last entry since this is an artificial one to finish the for-loop above.
for i in dance_name_ids[:-1]:
    df_finals.drop(i, inplace=True)

# Replace the couple names by only their number.
df_finals[[C.NR]] = df_finals.iloc[:, 0].str.extract(r'(\d+)\D*', expand = True)
df_finals[C.NR].iloc[1] = C.NR
df_finals = df_finals.iloc[:, 1:]

# Replace the adjudicator names by only their reference letter.
for i, value in enumerate(df_finals.iloc[1, :]):
    if value == "1.": 
        break

    df_finals.iloc[1, i] = value[:1]

# Delete the first line which only denotes where the adjudicators and results columns are - same as the second row does.
df_finals = df_finals.iloc[1:]
# Set the now first row as header.
df_finals.columns = df_finals.iloc[0]
df_finals = df_finals[1:]

# Use the couple number as index column.
df_finals.set_index(C.NR, drop=True, inplace=True)
df_finals.index.name = C.NR

df_dict[data_dfs.finals] = df_finals
df_dict[data_dfs.finals].to_csv(Path(competition_dir, c.filenames.finals.value))
df_dict[data_dfs.finals]

AttributeError: 'str' object has no attribute 'value'

## Processing of the qualifications table

In [ ]:
row_id_marks: int = 0
row_id_sums: int  = 1
row_id_quali: int = 2
row_id_round_offset: int = 3

In [ ]:
df_qualifications = df_dict[data_dfs.qualifications][0]
n: int = len(df_dict[data_dfs.qualifications])
if n > 1:
    for i in range(1, n):
        # Drop the first column since it is already present from the first dataframe.
        df_tmp = df_dict[data_dfs.qualifications][i].iloc[:, 1:]
        # Append the dataframe to the rest.
        df_qualifications = pd.concat([df_qualifications, df_tmp], axis=1)

# Delete the first row since it does not really contain information
df_qualifications = df_qualifications[1:]
# Because of the concatenation the column indices could be messed up. Reset them to a range.
df_qualifications.columns = range(len(df_qualifications.columns))

# Delete all rows where all values are NaN.
df_qualifications.dropna(axis=1, how='all', inplace=True)

# Depending on the size of the tournament, the names of the competitors are perhaps repeated.
# In this case frop this row since it is redundant.
df_qualifications.drop(df_qualifications.loc[df_qualifications.iloc[:, 0] == 'Startnummer'].index, inplace=True)


# Replace the adjudicator names by only their reference letter.
# Replace the couple names by only their number.
couple_nrs = df_qualifications.iloc[0, :].str.extract(r'(\d+)\D*')
couple_nrs_list = couple_nrs[0].tolist()
couple_nrs_list[0] = C.NR
df_qualifications.columns = couple_nrs_list
# Drop the first row since it contains only the names of the couples and is thus 
# equivalent to the new column names.
df_qualifications = df_qualifications[1:]
df_qualifications

,Nr.,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,66
2,A) Petra Matschullat-Horn B) Christoph Losensk...,1 1 5 1 2 5 0,4 3 5 4 5 5 5,4 4 5 4 5 4 3,2 0 0 1 0 0 2,4 5 5 5 4 5 1,5 5 5 5 5 5 5,5 5 5 5 5 5 5,1 1 1 2 0 4 1,1 0 0 1 3 0 4,...,5 5 4 5 5 5 5,0 4 4 0 3 1 5,5 5 5 5 5 5 5,0 0 0 0 0 0 0,5 5 5 5 5 5 5,5 5 5 5 5 2 5,1 4 0 2 3 0 2,5 0 4 0 0 0 0,0 4 1 3 1 4 0,5 4 1 5 4 5 4
3,Ergebnis der Vorrunde,15,31,29,5,29,35,35,10,9,...,34,17,35,0,35,32,12,9,13,28
4,Qualifiziert für die 1. Zwischenrunde,X,X,X,NaN,X,X,X,NaN,NaN,...,X,X,X,NaN,X,X,NaN,NaN,NaN,X
5,A) Petra Matschullat-Horn B) Christoph Losensk...,0 0 0 0 0 0 0,3 1 4 4 5 5 5,0 3 0 0 1 0 0,NaN,0 0 2 2 0 1 3,5 5 5 5 5 5 5,5 5 5 4 5 5 5,NaN,NaN,...,3 5 4 5 3 5 1,0 0 0 0 1 0 3,5 2 5 0 4 4 3,NaN,5 5 5 5 3 5 5,4 4 0 5 2 0 0,NaN,NaN,NaN,0 0 0 0 1 0 0
6,Ergebnis der 1. Zwischenrunde,0,27,4,NaN,8,35,34,NaN,NaN,...,26,4,23,NaN,33,15,NaN,NaN,NaN,1
7,Qualifiziert für die Endrunde,NaN,X,NaN,NaN,NaN,X,X,NaN,NaN,...,X,NaN,X,NaN,X,NaN,NaN,NaN,NaN,NaN
8,Addition Endrunde,NaN,210,NaN,NaN,NaN,50,170,NaN,NaN,...,245,NaN,275,NaN,100,NaN,NaN,NaN,NaN,NaN
10,Platz von Platz bis,12,4,9 10,19,8,1,3,15,16 17,...,5,9 10,6,20,2,7,14,16 17,13,11
11,Aufstiegspunkte,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [ ]:
# The raw data has been parsed from the qualifications table.
# Now store the tables for the individual rounds in a table each.
df_dict[data_dfs.qualifications] = []

for r in range(nr_rounds):
    # Split the values and map them to new columns. These correspond to the marks of the adjudicaotrs in order.
    round_marks = df_qualifications.iloc[row_id_marks + r * row_id_round_offset, 1:]
    round_marks.replace(float('Nan'), str('- ' * nr_adjudicators)[:-1], inplace=True)
    df_round_marks = round_marks.str.split(' ', expand=True)
    df_round_marks.columns = adjudicator_ids
    df_round_marks[C.NR] = list(df_qualifications.columns)[1:]
    df_round_marks.set_index(C.NR drop=True, inplace=True)
    df_round_marks.index.name = C.NR
    # df_round_marks.drop(columns=[C.NR.value])

    # Etract the sums.
    sums = df_qualifications.iloc[row_id_sums + r * row_id_round_offset, 1:]
    sums.replace(float('NaN'), -1, inplace=True)
    df_round_marks[C.SUM] = sums

    # Extract the qualification indications, and replace the values by binary ones.
    quali = df_qualifications.iloc[row_id_quali + r * row_id_round_offset, 1:]
    quali.replace(float('NaN'), False, inplace=True)
    quali.replace('X', True, inplace=True)
    # Add the sums as column to the DataFrame.
    df_round_marks[C.QUALI] = quali

    df_dict[data_dfs.qualifications].append(df_round_marks)
    df_round_marks.to_csv(Path(competition_dir, c.filenames.quali_roundx.value.format(r)))

In [ ]:
df_dict[data_dfs.qualifications][0]

,A,B,C,D,E,F,G,Sum,Qualified
Nr.,,,,,,,,,
1,1,1,5,1,2,5,0,15,True
2,4,3,5,4,5,5,5,31,True
3,4,4,5,4,5,4,3,29,True
4,2,0,0,1,0,0,2,5,False
5,4,5,5,5,4,5,1,29,True
6,5,5,5,5,5,5,5,35,True
7,5,5,5,5,5,5,5,35,True
8,1,1,1,2,0,4,1,10,False
9,1,0,0,1,3,0,4,9,False


In [ ]:
df_dict[data_dfs.qualifications][-1]

,A,B,C,D,E,F,G,Sum,Qualified
Nr.,,,,,,,,,
1,0,0,0,0,0,0,0,0,False
2,3,1,4,4,5,5,5,27,True
3,0,3,0,0,1,0,0,4,False
4,-,-,-,-,-,-,-,-1,False
5,0,0,2,2,0,1,3,8,False
6,5,5,5,5,5,5,5,35,True
7,5,5,5,4,5,5,5,34,True
8,-,-,-,-,-,-,-,-1,False
9,-,-,-,-,-,-,-,-1,False


## Add further collected information to the general information table and save it

In [ ]:
additional_info[C.NR_ROUNDS] = nr_rounds + 1
additional_info[C.NR_ADJDCTRS] = nr_adjudicators
additional_info[C.NR_COUPLES] = nr_couples
df_additional_info = pd.DataFrame(list(additional_info.items()), columns=[C.CATEGORY, C.VALUE])

df_dict[data_dfs.general_info] = pd.concat([df_general_info, df_additional_info])
df_dict[data_dfs.general_info].to_csv(Path(competition_dir, c.filenames.general_info.value))
df_dict[data_dfs.general_info]

,Category,Value
1,Ausrichter,TSC Grün-Weiß Aquisgrana Aachen e.V.
2,Turnierleiter,Guido Gehlhaar TSC Grün-Weiß Aquisgrana Aachen
3,Beisitzer,Ilona Gehlhaar TSC Grün-Weiß Aquisgrana Aachen
4,Protokoll,Tobias Hock TSC Grün-Weiß Aquisgrana Aachen
5,Turnierbüro,Ursula Belten TSC Grün-Weiß Aquisgrana Aachen ...
6,Musik,Kerstin Fasel TSC Grün-Weiß Aquisgrana Aachen ...
0,Date,15.06.2019
1,Title,DTV-Ranglistenturnier Sen I S Std - Aachen
2,Class,Sen.I S Standard
3,Nr. Rounds + Final,3
